# Face Skin Condition Analysis - Colab Training
## Step-by-step GPU training with proper setup

**Prerequisites:**
- Upload `face-skin-analysis.tar.gz` (194MB) to your Google Drive root
- Select GPU runtime: Runtime → Change runtime type → GPU (T4)

## 🔗 Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print('✅ Google Drive mounted!')

## 📦 Step 2: Extract Project Archive

**Important:** Make sure you've uploaded `face-skin-analysis.tar.gz` to your Google Drive root!

In [ ]:
import os

# Check if archive exists
archive_path = '/content/drive/MyDrive/face-skin-analysis.zip'
project_path = '/content/Face-skin-analysis'

if not os.path.exists(archive_path):
    print('❌ ERROR: archive not found in Google Drive!')
    print('Please upload the zip to your Google Drive root folder.')
else:
    print(f'✅ Archive found: {archive_path}')
    
    # Create project directory if it doesn't exist
    os.makedirs(project_path, exist_ok=True)
    
    # Check if already extracted (look for key files)
    if not os.path.exists(os.path.join(project_path, 'configs')):
        print('📦 Extracting project... (this may take 1-2 minutes)')
        # Use unzip for the .zip archive
        !unzip -q {archive_path} -d {project_path}
        print('✅ Extraction complete!')
    else:
        print('✅ Project already extracted!')
    
    # Verify extraction
    print('\n📂 Project contents:')
    !ls -la {project_path}

## 🔧 Step 3: Navigate to Project & Install Dependencies

In [ ]:
# Change to project directory
import os
import sys

project_path = '/content/Face-skin-analysis'
os.chdir(project_path)
print(f'📂 Current directory: {os.getcwd()}')

# Add to Python path (use the same path)
sys.path.insert(0, project_path)

print('✅ Python path configured!')

In [ ]:
# 🔧 Update config base_dir for Colab to use /content
import yaml
cfg_path = '/content/Face-skin-analysis/configs/config.yaml'
with open(cfg_path, 'r') as f:
    cfg = yaml.safe_load(f)
cfg.setdefault('environment', {}).setdefault('colab', {})['base_dir'] = '/content/Face-skin-analysis'
with open(cfg_path, 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False, indent=2)
print('✅ Updated configs/config.yaml base_dir to /content/Face-skin-analysis')

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

print('✅ Dependencies installed!')

## 🎮 Step 4: Verify GPU & Environment

In [ ]:
import torch

# Check GPU
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB')
else:
    print('⚠️ WARNING: No GPU detected! Go to Runtime → Change runtime type → Select GPU')

# Now import project modules
from utils.environment import get_environment

env = get_environment()
print(f'\n✅ Environment: {env}')

## 📊 Step 5: Verify Data

In [ ]:
# Check processed data
print('📊 Checking processed data...')
!ls -lh data/processed/

print('\n📄 Data splits:')
!wc -l data/processed/train.csv data/processed/val.csv data/processed/test.csv

print('\n✅ Data verification complete!')

## 🔍 Step 5a: Check Skin Tone Distribution (Diagnostic)

In [ ]:
import pandas as pd

# Check skin tone distribution in splits
print("📊 Skin Tone Distribution Analysis\n")

for split in ['train', 'val', 'test']:
    df = pd.read_csv(f'data/processed/{split}.csv')
    print(f"{split.upper()} SET:")
    print(f"  Total samples: {len(df)}")
    
    if 'skin_tone' in df.columns:
        counts = df['skin_tone'].value_counts()
        print(f"  Skin tone values:\n{counts}\n")
        
        # Check if any valid Fitzpatrick values exist
        valid_fitz = df['skin_tone'].isin(['1', '2', '3', '4', '5', '6', 1, 2, 3, 4, 5, 6])
        print(f"  Valid Fitzpatrick labels: {valid_fitz.sum()}/{len(df)}\n")
    else:
        print("  ❌ No skin_tone column found\n")

print("\n⚠️ If all values are 'unknown', balanced sampling won't work!")
print("Solution: Re-run preprocessing with Fitzpatrick17k metadata extraction")

## 🔧 Step 5b: Fix Skin Tones (Run if all are 'unknown')

**Only run this if Step 5a showed all skin_tone values as 'unknown'**

This will:
1. Re-extract Fitzpatrick17k metadata to get real skin tone labels (1-6)
2. Regenerate metadata.csv and train/val/test splits with skin tones
3. Enable balanced sampling for fairness

In [ ]:
# WARNING: This recreates metadata and splits (loses current train/val/test split)
# Only run if you need skin tone labels for fairness analysis

import subprocess
import sys

print("🔧 Regenerating metadata with Fitzpatrick skin tone labels...")
print("⚠️ This will create new train/val/test splits!\n")

# Run preprocessing to extract skin tones from Fitzpatrick17k metadata
result = subprocess.run(
    [sys.executable, "preprocessing/run.py"],
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

# Verify skin tones were extracted
print("\n📊 Verifying skin tone extraction...")
df = pd.read_csv('data/processed/train.csv')
counts = df['skin_tone'].value_counts()
print(f"Skin tone distribution:\n{counts}")

valid_fitz = df['skin_tone'].isin(['1', '2', '3', '4', '5', '6'])
print(f"\n✅ Valid Fitzpatrick labels: {valid_fitz.sum()}/{len(df)}")

if valid_fitz.sum() > 0:
    print("\n🎉 Success! Balanced sampling will now work in training.")
else:
    print("\n❌ Still no valid skin tones. Check Fitzpatrick17k dataset.")

## 🧪 Step 6: Test Data Loaders

In [ ]:
from training.data_loader import create_data_loaders
from utils.config_loader import load_config

# Load config
config = load_config('configs/config.yaml')

# Create data loaders (returns a dictionary)
data_loaders = create_data_loaders(config)
train_loader = data_loaders['train']
val_loader = data_loaders['val']
test_loader = data_loaders['test']

print(f'✅ Train batches: {len(train_loader)}')
print(f'✅ Val batches: {len(val_loader)}')
print(f'✅ Test batches: {len(test_loader)}')

# Test a batch
images, labels = next(iter(train_loader))
print(f'\n📦 Batch shape: {images.shape}')
print(f'📦 Labels shape: {labels.shape}')
print(f'🎯 Device: {images.device}')

print('\n✅ Data loaders working correctly!')

## 🔐 Step 7: Setup MLflow (Optional - Skip for now)

In [ ]:
import os

# Option 1: Disable MLflow tracking (recommended for first run)
os.environ['MLFLOW_TRACKING_URI'] = ''
print('✅ MLflow tracking disabled (training will be faster)')

# Option 2: Enable DagsHub tracking (uncomment if you have credentials)
# os.environ['MLFLOW_TRACKING_USERNAME'] = 'your-dagshub-username'
# os.environ['MLFLOW_TRACKING_PASSWORD'] = 'your-dagshub-token'
# print('✅ MLflow tracking enabled')

## 🚀 Step 8: Start Training!

This will take 2-3 hours on a T4 GPU. You can monitor progress in the output below.

In [ ]:
# Train the model
!python training/train.py

# Note: Training progress will show in the output below
# Checkpoints are automatically saved to outputs/checkpoints/

## 📈 Step 9: Monitor Training (Run these periodically)

In [ ]:
# Check GPU usage
!nvidia-smi

In [ ]:
# List saved checkpoints
!ls -lh outputs/checkpoints/

## 📊 Step 10: Evaluate Model

In [ ]:
# Run evaluation with bias analysis
!python evaluation/evaluate.py

print('\n✅ Evaluation complete!')
print('Results saved to outputs/evaluation/')

## 📊 Step 10a: Debug Bias Analysis (Diagnostic)

**Run this to diagnose why bias metrics show 0.0000**

In [ ]:
import pandas as pd
import numpy as np

# Load test CSV
test_df = pd.read_csv('data/processed/test.csv')

print("🔍 Checking Test Set Skin Tone Mapping\n")

# Check raw skin_tone values
print("Raw skin_tone distribution:")
print(test_df['skin_tone'].value_counts())

# Map to groups using config logic
tone_map = {'1': 'light', '2': 'light', '3': 'medium', '4': 'medium', '5': 'dark', '6': 'dark'}

def to_group(x):
    x_clean = str(x).strip()
    return tone_map.get(x_clean, 'unknown')

test_df['tone_group'] = test_df['skin_tone'].apply(to_group)

print("\nMapped to groups:")
print(test_df['tone_group'].value_counts())

# Check if any positives exist per group
print("\n📊 Positives per skin tone group:")
for group in ['light', 'medium', 'dark']:
    group_df = test_df[test_df['tone_group'] == group]
    print(f"\n{group.upper()} (n={len(group_df)}):")
    print(f"  has_acne: {group_df['has_acne'].sum()}")
    print(f"  has_pigmentation: {group_df['has_pigmentation'].sum()}")
    print(f"  has_wrinkles: {group_df['has_wrinkles'].sum()}")

# The issue: bias_analysis averages across ALL conditions using macro average
# If some groups have 0 positives for a condition, division by zero → 0.0
print("\n⚠️ If a group has 0 positives for ANY condition, macro avg precision/recall = 0.0")
print("This is a sklearn limitation with multi-label classification")

## ✅ Bias Analysis Fix Applied

**Problem:** Previous evaluation showed all bias metrics as 0.0000 despite valid skin tone labels.

**Root Cause:** Using `average="macro"` with multi-label classification returns 0.0 when any condition has zero positives in a group.

**Solution:** Changed to `average="samples"` which is appropriate for multi-label classification.

**Expected Results:**
- Non-zero precision, recall, F1 for all skin tone groups
- Per-condition breakdown showing performance by group
- New visualization: `per_condition_metrics.png` with detailed comparisons

**See:** `docs/BIAS_ANALYSIS_FIX.md` for complete technical details.

## 📥 Step 11: Download Results

All outputs are saved in Google Drive. You can access them directly from Drive or download specific files below:

In [ ]:
from google.colab import files

# Download trained model
print('Downloading best model...')
files.download('outputs/checkpoints/best_model.pth')

# Download evaluation metrics (if available)
import os
if os.path.exists('outputs/evaluation/metrics.json'):
    print('Downloading metrics...')
    files.download('outputs/evaluation/metrics.json')

## 🎉 Training Complete!

**What you've accomplished:**
- ✅ Trained a skin condition detection model on GPU
- ✅ Evaluated performance with bias analysis
- ✅ Model saved to Google Drive

**Files in Google Drive:**
```
Face-skin-analysis/
├── outputs/
│   ├── checkpoints/best_model.pth
│   ├── checkpoints/last_model.pth
│   ├── evaluation/metrics.json
│   └── logs/training.log
```

**Next steps:**
1. Review evaluation metrics
2. Test model locally with `inference/predict.py`
3. Export to ONNX: `python onnx/export.py`

---

**💡 Tip:** All files persist in Google Drive and can be accessed from your local machine via Google Drive Desktop or web interface.